In [1]:
from pyscf.lib import krylov
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [54]:
mol=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", unit="Bohr", basis= "sto-3g")
mf=scf.RHF(mol)
mf.scf()
dL=.01
mf1=fc(mf,[dL,-dL])
mf1.scf()
mf2=fc(mf,[-dL,dL])
mf2.scf()
dV=DeltaV(mol,[dL,-dL])
h2=mf.get_veff()
dh2=(mf1.get_veff()-mf2.get_veff())/2
J,K=mf.get_jk()
g_ijkl=mf.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
nocc= mol.nelec[0]


C=mf.mo_coeff
nmo=C.shape[0]
nvir=nmo-nocc
S=mf.get_ovlp()
e=mf.mo_energy
E=np.diag(e)
O=np.diag(mf.mo_occ)
C1=mf1.mo_coeff
P=mf.make_rdm1()
P1=mf1.make_rdm1()
P2=mf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/dL**2
dC=(abs(C1)*C/abs(C)-C)
dV_mo=C.T@dV@C

converged SCF energy = -111.199724042754
converged SCF energy = -111.127037547302
converged SCF energy = -111.272703385098


Initialize <pyscf.gto.mole.Mole object at 0x7f19c80bb048> in <pyscf.scf.hf.RHF object at 0x7f19fc5d2b38>


In [55]:
def test(u_): # check density matrices comparing to the one ontained by FD
    ut=np.zeros((nmo,nmo))
    ut[nocc:,:nocc]=u_.reshape(nvir,nocc)
    ut[:nocc,nocc:]=-u_.reshape(nvir,nocc).T
    dP_app=C@(ut@O-O@ut)@C.T    
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))
    print(np.linalg.norm(dP_app-dP))

In [56]:
em1,em2=np.meshgrid(e,e)
emeshvo=(em1-em2)[nocc:,:nocc]
emeshf=emeshvo.flatten()
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)

G_cphfvovo=G_cphf.copy()[nocc:,:nocc,nocc:,:nocc]    #aibj

In [57]:
dVvo=(dV_mo[nocc:,:nocc])
dVf=dVvo.flatten()

In [58]:
def emul(_u):
    return _u*emeshf-_u #from U gives dV

In [59]:
def gmul(_u):
    return np.einsum('ijkl,kl->ij',G_cphfvovo,_u.reshape(nvir,nocc)).flatten()

In [60]:
def amul(_u):
    return emul(_u)-gmul(_u)

In [61]:
uk=krylov(amul,dVf, max_cycle=100)

In [62]:
np.max(amul(uk)+uk-dVf)

6.84001466577655e-15

In [63]:
np.linalg.norm(amul(uk)+uk-dVf)

1.2242083042477324e-14

In [64]:
np.max(dVf)

0.005638559281012129

In [65]:
test(uk)

0.017892688180805253
0.017892788637837286
2.5716355479147853e-07


In [14]:
help(krylov)

Help on function krylov in module pyscf.lib.linalg_helper:

krylov(aop, b, x0=None, tol=1e-10, max_cycle=30, dot=<function dot at 0x7f19fc255d08>, lindep=1e-15, callback=None, hermi=False, max_memory=2000, verbose=2)
    Krylov subspace method to solve  (1+a) x = b.  Ref:
    J. A. Pople et al, Int. J.  Quantum. Chem.  Symp. 13, 225 (1979).
    
    Args:
        aop : function(x) => array_like_x
            aop(x) to mimic the matrix vector multiplication :math:`\sum_{j}a_{ij} x_j`.
            The argument is a 1D array.  The returned value is a 1D array.
        b : a vector or a list of vectors
    
    Kwargs:
        x0 : 1D array
            Initial guess
        tol : float
            Tolerance to terminate the operation aop(x).
        max_cycle : int
            max number of iterations.
        lindep : float
            Linear dependency threshold.  The function is terminated when the
            smallest eigenvalue of the metric of the trial vectors is lower
            t